![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xg
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV

import joblib
import werkzeug
from werkzeug.utils import cached_property
from flask import Flask

try:
    from flask_restplus import Api, Resource, fields
except ImportError:
    import werkzeug
    werkzeug.cached_property = werkzeug.utils.cached_property
    from flask_restplus import Resource, Api, fields

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

In [ ]:
XTotal = dataTraining.loc[:,dataTraining.columns!="Price"]
yTotal = dataTraining.loc[:,dataTraining.columns=="Price"]
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.33, random_state=0)

## Exploración de Datos

In [ ]:
##Buscamos valores Nulos en las variables.
XTrain.isnull().sum()

In [ ]:
print(XTrain['Year'].describe())
XTrain['Year'].value_counts()

In [ ]:
print(XTrain['Mileage'].describe())
XTrain['Mileage'].value_counts()

In [ ]:
print(XTrain['State'].describe())
XTrain['State'].value_counts()

In [ ]:
print(XTrain['Make'].describe())
XTrain['Make'].value_counts()

In [ ]:
print(XTrain['Model'].describe())
XTrain['Model'].value_counts()

## Transformación de Datos

In [ ]:
XTrain.State = XTrain.State.str.replace(' ', '')
XTrain['YearsBetween']=2022-XTrain['Year']
XTrain['MileagebyYear']=XTrain['Mileage']/XTrain['YearsBetween']
State=pd.get_dummies(XTrain["State"], prefix='State')
XTrain=pd.concat([XTrain, State], axis=1)
Make=pd.get_dummies(XTrain["Make"], prefix='Make')
XTrain=pd.concat([XTrain, Make], axis=1)
Model=pd.get_dummies(XTrain["Model"], prefix='Model')
XTrain=pd.concat([XTrain, Model], axis=1)

XTrain=XTrain.drop(["State","Make","Model","Make_Freightliner"], axis=1)
#ss = StandardScaler()
#XTrain = ss.fit_transform(XTrain)
XTrain

In [ ]:
XTest.State = XTest.State.str.replace(' ', '')
XTest['YearsBetween']=2022-XTest['Year']
XTest['MileagebyYear']=XTest['Mileage']/XTest['YearsBetween']
State=pd.get_dummies(XTest["State"], prefix='State')
XTest=pd.concat([XTest, State], axis=1)
Make=pd.get_dummies(XTest["Make"], prefix='Make')
XTest=pd.concat([XTest, Make], axis=1)
Model=pd.get_dummies(XTest["Model"], prefix='Model')
XTest=pd.concat([XTest, Model], axis=1)

XTest=XTest.drop(["State","Make","Model","Make_Freightliner"], axis=1)
#XTest = ss.transform(XTest)
XTest

## Ajuste de Modelo Sin Calibración

In [ ]:
# Instantiation
xgb_r = xg.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.1, learning_rate = 0.1, max_depth = 5, verbosity = 0)

# Fitting the model
xgb_r.fit(XTrain, yTrain)
  
# Predict the model
pred = xgb_r.predict(XTest)
  
# RMSE Computation
rmse_base = np.sqrt(MSE(yTest, pred))
print("RMSE : % f" %(rmse_base))

## Proceso de Calibración

### ¡Advertencia!
### Este proceso puede tomar más de 4hrs

In [ ]:
# learning_rate calibration

learning_rate_range = np.linspace(0.01, 0.5, num=10)
accuracy_scores = []

for learning_rate in learning_rate_range:
    
    clf = xg.XGBRegressor(objective='reg:squarederror',learning_rate=learning_rate, verbosity = 0 )

    # Fitting the model
    clf.fit(XTrain, yTrain)

    # Predict the model
    pred = clf.predict(XTest)

    # RMSE Computation
    rmse=np.sqrt(MSE(yTest, pred))
    accuracy_scores.append(rmse)
    print(learning_rate,rmse)
plt.plot(learning_rate_range, accuracy_scores)
plt.xlabel('learning_rate')
plt.ylabel('RMSE')
plt.show()
print(" Alcanzamos el minimo RMSE con learning_rate = " + str(learning_rate_range[np.argmin(accuracy_scores)]))

In [ ]:
# gamma calibration
gamma_range = np.linspace(0.01, 0.5, num=10)
accuracy_scores = []
for gamma in gamma_range:
    
    clf = xg.XGBRegressor(objective='reg:squarederror',learning_rate=0.5,gamma=gamma, verbosity = 0 )
    
    # Fitting the model
    clf.fit(XTrain, yTrain)

    # Predict the model
    pred = clf.predict(XTest)

    # RMSE Computation
    rmse=np.sqrt(MSE(yTest, pred))
    accuracy_scores.append(rmse)
    print(gamma,rmse)
    
plt.plot(gamma_range, accuracy_scores)
plt.xlabel('gamma')
plt.ylabel('RMSE')
plt.show()
print(" Alcanzamos el minimo RMSE con gamma = " + str(gamma_range[np.argmin(accuracy_scores)]))

In [ ]:
# colsample_bytree calibration
colsample_bytree_range = np.linspace(0.01, 0.5, num=10)
accuracy_scores = []
for colsample_bytree in colsample_bytree_range:
    
    clf = xg.XGBRegressor(objective ='reg:linear',learning_rate=0.5 ,gamma=0.01, colsample_bytree=colsample_bytree, verbosity = 0 )
    
    # Fitting the model
    clf.fit(XTrain, yTrain)

    # Predict the model
    pred = clf.predict(XTest)

    # RMSE Computation
    rmse=np.sqrt(MSE(yTest, pred))
    accuracy_scores.append(rmse)
    print(colsample_bytree,rmse)
    
print(colsample_bytree_range[np.argmin(accuracy_scores)])
plt.plot(colsample_bytree_range, accuracy_scores)
plt.xlabel('colsample_bytree')
plt.ylabel('RMSE')
plt.show()

In [ ]:
# colsample_bytree calibration
max_depth_range = estimator_range = range(1, 10)

accuracy_scores = []
for max_depth in max_depth_range:
    
    clf = xg.XGBRegressor(objective='reg:squarederror',learning_rate=0.5 ,gamma=0.01, colsample_bytree=0.5,max_depth=max_depth, verbosity = 0 )
    
    # Fitting the model
    clf.fit(XTrain, yTrain)

    # Predict the model
    pred = clf.predict(XTest)

    # RMSE Computation
    rmse=np.sqrt(MSE(yTest, pred))
    accuracy_scores.append(rmse)
    print(max_depth,rmse)
    
print(max_depth_range[np.argmin(accuracy_scores)])
plt.plot(max_depth_range, accuracy_scores)
plt.xlabel('max_depth')
plt.ylabel('RMSE')
plt.show()

In [ ]:
# colsample_bytree calibration
alpha_range = estimator_range = range(1, 10)


accuracy_scores = []
for alpha in alpha_range:
    
    clf = xg.XGBRegressor(objective='reg:squarederror',learning_rate=0.5 ,gamma=0.01, colsample_bytree=0.5,max_depth=max_depth,alpha=alpha ,verbosity = 0 )
    
    # Fitting the model
    clf.fit(XTrain, yTrain)

    # Predict the model
    pred = clf.predict(XTest)

    # RMSE Computation
    rmse=np.sqrt(MSE(yTest, pred))
    accuracy_scores.append(rmse)
    print(alpha,rmse)
    
print(max_depth_range[np.argmin(accuracy_scores)])
plt.plot(max_depth_range, accuracy_scores)
plt.xlabel('alpha')
plt.ylabel('RMSE')
plt.show()

## Entrenar  Modelo Calibrado

In [ ]:
# Modelo calibrado
clf = xg.XGBRegressor(learning_rate=0.5,gamma=0.01, colsample_bytree=0.5, max_depth=9,alpha=1,verbosity = 0 )    
# Fitting the model
clf.fit(XTrain, yTrain)
# Predict the model
pred = clf.predict(XTest)
# RMSE Computation
rmse_cal=np.sqrt(MSE(yTest, pred))
print('RMSE:',rmse_cal)

## Guardar el Modelo Calibrado

In [ ]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'modelo_precio_autos_calibrado.pkl', compress=3)

In [ ]:
# Importar modelo y predicción
from predice_precio_model_deployment import predice_precio

# Predicción de precio  de  un auto
predice_precio(2017,9913,'FL','Jeep','Wrangler') 
#34995

## Resultados

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title("Comparación de Tiempos de Ejecución entre Modelos (RMSE)")
ejeX = ['XGBoost','XGBoost Cal.']
ejeY = [float(round(rmse_base,4)),float(round(rmse_cal,4))]
ax.bar(ejeX,ejeY)
def addlabels(x,y,plotP):
    for i in range(len(x)):
        plotP.text(i,y[i],y[i])
addlabels(ejeX,ejeY,plt)
plt.show()

## Perfilando datos de Test

In [ ]:
# Visualización datos de test
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)
dataTesting['YearsBetween']=2022-dataTesting['Year']
dataTesting['MileagebyYear']=dataTesting['Mileage']/dataTesting['YearsBetween']
State=pd.get_dummies(dataTesting["State"], prefix='State')
dataTesting=pd.concat([dataTesting, State], axis=1)
Make=pd.get_dummies(dataTesting["Make"], prefix='Make')
dataTesting=pd.concat([dataTesting, Make], axis=1)
Model=pd.get_dummies(dataTesting["Model"], prefix='Model')
dataTesting=pd.concat([dataTesting, Model], axis=1)

dataTesting=dataTesting.drop(["State","Make","Model"], axis=1)
pred = clf.predict(dataTesting)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
yPred=pd.DataFrame(pred,columns=['Price'])
# Exportamos a un archivo los resultados
yPred.to_csv('test_submission.csv', index_label='ID')
yPred.head()

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [9]:
from predice_precio_model_deployment import predice_precio

# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='API de Predicción de Precios de Autos',
    description='API de Predicción de Precios de Autos')

ns = api.namespace('predict', 
     description='Regresión de Precio de Auto')

# Definición argumentos o parámetros de la API
parser = api.parser()

parser.add_argument(
    'Year', 
    type=int, 
    required=True, 
    help='Año del modelo', 
    location='args')

parser.add_argument(
    'Mileage', 
    type=int, 
    required=True, 
    help='Millas recorridas', 
    location='args')

parser.add_argument(
    'State', 
    type=str, 
    required=True, 
    help='Estado de procedencia', 
    location='args')

parser.add_argument(
    'Make', 
    type=str, 
    required=True, 
    help='Marca de la ensambladora de auto', 
    location='args')

parser.add_argument(
    'Model', 
    type=str, 
    required=True, 
    help='Sub marca del auto', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [10]:
# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        print(args)
        return {
         "result": predice_precio(args['Year'],args['Mileage'],args['State'],args['Make'],args['Model'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2022 18:30:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 18:30:17] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 18:30:53] "GET /predict/?Year=2017&Mileage=1983&State=FL&Make=Jeep&Model=Wrangler HTTP/1.1" 200 -


{'Year': 2017, 'Mileage': 1983, 'State': 'FL', 'Make': 'Jeep', 'Model': 'Wrangler'}
